In [75]:
import requests
import re
import os
from bs4 import BeautifulSoup
import pickle
import bz2file as bz2
from urllib.request import urlopen
from urllib.parse import quote
import statistics as stats
import pickle
import nltk
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
import matplotlib.pyplot as plt
import community as community_louvain
from collections import Counter
from wordcloud import WordCloud
from fa2_modified import ForceAtlas2
import matplotlib.colors as mcolors
import math

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [76]:
baseUrl = "https://gameofthronesfanon.fandom.com/"

In [77]:
def save(title, data):
    with bz2.BZ2File(f"{title}.pbz2", "w") as f:
        pickle.dump(data, f)

In [78]:
def load(title):
    with bz2.BZ2File(f"{title}.pbz2", "r") as f:
        return pickle.load(f)

In [79]:
def fetch_page(url):
    response = requests.get(url)
    response.raise_for_status()
    return response.text

In [80]:
def save_to_file(content, filename):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(content)

In [81]:
# Get all urls for the characters
urls = []
for char in ["a", "b", "c", "d", "e", "f", "g", "h", "i", "j", "k", "l", "m", "n", "o", "p", "q", "r", "s", "t", "u", "v", "w", "x", "y", "z"]:
     url = f"{baseUrl}wiki/Category:Characters?from={char}"
     urls.append(url)

In [82]:
# Get all characters from the Game of Thrones Fanon Wiki
if not os.path.exists('../data/all_characters.html'):   
    all_content = ""
    for url in urls:
        content = fetch_page(url)
        if content:
            all_content += content
        else:
            print(f"Failed to fetch {url}")

    save_to_file(all_content, '../data/all_characters.html')

In [83]:
class BiographicalInfo:
    def __init__(self, name, birth_year, birth_place):
        self.name = name
        self.birth_year = birth_year
        self.birth_place = birth_place
    
    def __repr__(self):
        return f"{self.name} was born in {self.birth_place} in {self.birth_year}"

In [84]:
class PoliticalInfo:
    def __init__(self, houses, titles):
        self.houses = houses
        self.titles = titles
    
    def __repr__(self):
        return f"{self.titles} of {self.houses}"

In [85]:
class PersonalInfo:
    def __init__(self, aka, culture, religon, father, mother, spouses, issues, siblings):
        self.aka = aka
        self.culture = culture
        self.religon = religon
        self.father = father
        self.mother = mother
        self.spouses = spouses
        self.issues = issues
        self.siblings = siblings
    
    def __repr__(self):
        return f"{self.aka} {self.culture} {self.religon} {self.father} {self.mother} {self.spouses} {self.issues} {self.siblings}"

In [86]:
class Character:
    def __init__(self, name, url, html, text):
        self.name = name
        self.url = url
        self.html = html
        self.text = text
        self.references = []
        self.biographical_info = None
        self.personal_info = None
        self.political_info = None
        self.sentiment = None
        self.tfidf = None

    def __repr__(self):
        return f"Character(name={self.name}, url={self.url})"

In [87]:
def get_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    text = soup.get_text(separator="\n", strip=True)
    return text

In [88]:
def get_html(url):
    content = fetch_page(url)
    return content

In [89]:
def create_character(name, url):
    text = get_text(url)
    html = get_html(url)
    character = Character(name, url, html, text)
    return character

In [90]:
def get_characters_from_matches(matches):
    skip = ["Category:", "User:"]
    characters = []
    for href, title in matches:
        if any(s in title for s in skip):
            continue
        try:
            character = create_character(title, baseUrl + href)
            characters.append(character)
        except Exception as e:
            pass

    return characters

In [91]:
def get_all_character_matches_of_html():
    with open('../data/all_characters.html', 'r', encoding='utf-8') as file:
        all_characters_html = file.read()
        pattern = r'<a\s+href="([^"]+)"\s+title="([^"]+)"'
        matches = re.findall(pattern, all_characters_html)
        return matches

In [92]:
# get all characters from the html file and add the text and raw html to the character
path = '../data/characters_1'
characters = []
if os.path.exists(f"{path}.pbz2"):
    characters = load(path)
else:    
    matches = get_all_character_matches_of_html()
    characters = get_characters_from_matches(matches)
    save(path, characters)

In [93]:
# add references to the characters
pattern = r'<a\s+href="([^"]+)"'
all_character_urls = [character.url for character in characters]
for character in characters:
    matches = re.findall(pattern, character.html)
    for href in matches:
        url = baseUrl + href
        if url in all_character_urls and url != character.url:
            reference = [c for c in characters if c.url == url][0]
            character.references.append(reference)

In [94]:
class CharacterAttribute:
    def __init__(self, data_source_title, texts):
        self.data_source_title = data_source_title
        self.texts = texts
        self.hrefs = []

    def __repr__(self):
        return f"{self.data_source_title} {self.texts} {self.hrefs}"

In [95]:
def get_by_data_source(character, data_source_title, tag):
    soup = BeautifulSoup(character.html, 'html.parser')
    result = soup.find(attrs={"data-source": data_source_title})
    if result:
        if tag is None:
            results = result
        else:
            results = result.findAll(tag)
        texts = [result.text for result in results]
        characterAttribute = CharacterAttribute(data_source_title, texts)
        if tag == "a":
            hrefs = [result['href'] for result in results]
            characterAttribute.hrefs = hrefs
        return characterAttribute
    return None

In [96]:
def get_name(character):
    return get_by_data_source(character, "Title", None)

In [97]:
def get_birth_year(character):
    return get_by_data_source(character, "Birth", "div")

In [98]:
def get_birth_place(character):
    return get_by_data_source(character, "Birth", "a")

In [99]:
def get_titles(character):
    return get_by_data_source(character, "Titles", "a")

In [100]:
def get_houses(character):
    return get_by_data_source(character, "House", "a")

In [101]:
def get_AKA(character):
    return get_by_data_source(character, "AKA", "div")

In [102]:
def get_culture(character):
    return get_by_data_source(character, "Culture", "a")

In [103]:
def get_religion(character):
    return get_by_data_source(character, "Religion", "a")

In [104]:
def get_father(character):
    return get_by_data_source(character, "Father", "a")

In [105]:
def get_mother(character):
    return get_by_data_source(character, "Mother", "a")

In [106]:
def get_spouses(character):
    return get_by_data_source(character, "Spouse", "div")

In [107]:
def get_issues(character):
    return get_by_data_source(character, "Issue", "a")

In [108]:
def get_siblings(character):
    return get_by_data_source(character, "Siblings", "a")

In [109]:
def get_character_bibliographical_info(character):
    name = get_name(character)
    birth_place = get_birth_place(character)
    birth_year = get_birth_year(character)
    return BiographicalInfo(name, birth_year, birth_place)

In [110]:
def get_political_info(character):
    houses = get_houses(character)
    titles = get_titles(character)
    return PoliticalInfo(houses, titles)

In [111]:
def get_personal_info(character):
    aka = get_AKA(character)
    culture = get_culture(character)
    religion = get_religion(character)
    father = get_father(character)
    mother = get_mother(character)
    spouses = get_spouses(character)
    issues = get_issues(character)
    siblings = get_siblings(character)
    return PersonalInfo(aka, culture, religion, father, mother, spouses, issues, siblings)

In [112]:
# add attributes to the characters
path = '../data/characters_2'
if os.path.exists(f"{path}.pbz2"):
    characters = load(path)
else:   
    for character in characters:
        character.biographical_info = get_character_bibliographical_info(character)
        character.political_info = get_political_info(character)
        character.personal_info = get_personal_info(character) 
    save(path, characters)

In [113]:
path = "../data/word_happines"
if not os.path.exists(f"{path}.pbz2"):
    url = "https://hedonometer.org/api/v1/words/?format=json&wordlist__title=labMT-en-v2"

    limit = 1000 
    offset = 0 

    word_happiness_dict = {}

    while True:
        response = requests.get(url, params={"limit": limit, "offset": offset})
        
        if response.status_code == 200:
            data = response.json()
            
            for entry in data["objects"]:
                word_happiness_dict[entry["word"]] = entry["happs"]
            
            if data["meta"]["next"] is None:
                break  
            
            offset += limit
        else:
            print(f"Failed to retrieve data. Status code: {response.status_code}")
            break
    
    save(path, word_happiness_dict)
else:
    word_happiness_dict = load(path)

In [114]:
def get_tokens_for_sentiment(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens]
    return tokens

In [115]:
def calculate_sentiment(tokens):
    tokens = [token for token in tokens if token in word_happiness_dict]
    return sum(word_happiness_dict[token] for token in tokens) / len(tokens)

In [116]:
def add_sentiment_to_characters():
    for character in characters:
        tokens = get_tokens_for_sentiment(character.text)
        sentiment = calculate_sentiment(tokens)
        character.sentiment = sentiment

In [117]:
def get_tokens_for_tfidf(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha()]
    tokens = [word for word in tokens if word not in stop_words]
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return tokens

In [118]:
def get_tf_dict(characters):
    tf_dict = {}
    for character in characters:
        tokens = get_tokens_for_tfidf(character.text)
        tf_dict_inner = {}
        for token in tokens:
            if token not in tf_dict_inner:
                tf_dict_inner[token] = 1
            else:
                tf_dict_inner[token] += 1
        tf_dict[character] = tf_dict_inner
    return tf_dict

In [119]:
def get_idf_dict(tf_dict):
    token_counts = {}

    for _, tf_dict_inner in tf_dict.items():
        unique_tokens = set(tf_dict_inner.keys())
        for token in unique_tokens:
            if token not in token_counts:
                token_counts[token] = 1
            else:
                token_counts[token] += 1

    idf_dict = {}
    for token, count in token_counts.items():
        idf_dict[token] = math.log(len(characters) / count)
    
    return idf_dict

In [120]:
def add_tfidf_to_characters(tf_dict, idf_dict):
    for character, tf_dict_inner in tf_dict.items():
        tfidf_dict_inner = {}
        for token, count in tf_dict_inner.items():
            tfidf_dict_inner[token] = count * idf_dict[token]
        character.tfidf = sorted(tfidf_dict_inner.items(), key=lambda x: x[1], reverse=True)

In [122]:
path = '../data/characters_3'
if os.path.exists(f"{path}.pbz2"):
    characters = load(path)
else:   
    add_sentiment_to_characters()
    tf_dict = get_tf_dict(characters)
    idf_dict = get_idf_dict(tf_dict)
    add_tfidf_to_characters(tf_dict, idf_dict)
    save(path, characters)